# Explicación de la APi aplicada al negocio.

Esta API puede ser una herramienta poderosa para empresas y emprendedores en sectores relacionados con transporte, logística, o cualquier servicio basado en movilidad (como delivery o transporte de pasajeros). A continuación, te detallaré cómo esta API aporta valor al negocio:

1. Identificación de Zonas con Mayor Demanda
- Problema del negocio: Conductores y empresas a menudo no saben en qué zonas concentrar sus esfuerzos para maximizar ingresos.
- Solución con la API: 
    + La API identifica las zonas con mayor demanda para cada día de la semana, proporcionando información precisa sobre:
        - Zonas más activas.
        - Horas pico de actividad.
        - Ganancias promedio en esas zonas.
    + Impacto:
        - Permite a las empresas asignar sus recursos (conductores, vehículos) de manera más eficiente.
        - Reduce el tiempo de espera o inactividad para los conductores, maximizando los ingresos por hora.

2. Predicción de Ganancias Basada en el Contexto
- Problema del negocio: Los conductores y las empresas necesitan estimar ingresos futuros para planificar sus operaciones y horarios.
- Solución con la API:
    + Usando un modelo de Machine Learning, la API predice las ganancias promedio esperadas según:
        - Zona específica.
        - Hora del día.
        - Día de la semana.
    + Impacto:
        - Los conductores pueden planificar mejor sus horarios, enfocándose en momentos de alta rentabilidad.
        - Las empresas pueden usar estas predicciones para ajustar tarifas dinámicas en función de la demanda estimada.
3. Optimización de Rutas y Tiempos
- Problema del negocio: Los conductores a menudo recorren grandes distancias sin pasajeros, incurriendo en costos adicionales de combustible y tiempo.
- Solución con la API:
    + La API proporciona:
        - Información geográfica con enlaces directos a Google Maps para cada zona de alta demanda.
        - Distancias promedio recorridas en cada zona.
    + Impacto:
        - Optimiza rutas para minimizar desplazamientos sin pasajeros.
        - Ayuda a reducir costos operativos (combustible, desgaste del vehículo).
4. Planificación Estratégica para la Empresa
- Problema del negocio: Las empresas de transporte necesitan datos concretos para tomar decisiones estratégicas, como:
    + Dónde expandir operaciones.
    + Cómo ajustar tarifas.
    + En qué zonas priorizar campañas de marketing.
- Solución con la API:
    + Con datos históricos y predicciones:
        - Las empresas pueden identificar patrones de demanda estacional o diaria.
        - Permite optimizar precios en zonas y horarios de alta demanda para maximizar ingresos.
    + Impacto: Mejora la eficiencia operativa y la experiencia del cliente al reducir tiempos de espera y ajustar tarifas de manera más precisa.
5. Escenarios de Aplicación
    - Conductores independientes: Usan la API para decidir dónde y cuándo trabajar, asegurando que maximizan sus ingresos por hora.
    - Plataformas de transporte: Integran la API en su backend para ajustar tarifas dinámicas en tiempo real o sugerir zonas activas a los conductores.
    - Empresas de logística y delivery: Optimizan las rutas de entrega basadas en datos de alta actividad, reduciendo costos y tiempos de entrega.
    - Gobiernos o ciudades inteligentes: Analizan los datos para planificar infraestructura urbana, como estaciones de carga para vehículos eléctricos o mejoras en el transporte público.
6. Beneficios Clave para el Negocio
- Maximización de ingresos: Información basada en datos históricos y predicciones permite a los conductores y empresas aprovechar los momentos y lugares más rentables.
- Reducción de costos: Optimización de rutas y reducción de desplazamientos innecesarios.
- Mejor experiencia del cliente: Reducción de tiempos de espera y disponibilidad más eficiente.
- Toma de decisiones basada en datos: Estrategias operativas respaldadas por análisis de patrones históricos.

7. Conclusión
Esta API no solo resuelve problemas operativos en tiempo real, sino que también proporciona una base sólida para decisiones estratégicas a largo plazo. Implementarla puede significar una ventaja competitiva significativa en industrias donde cada minuto y cada kilómetro cuentan para el éxito del negocio.


In [55]:
from fastapi import FastAPI
import pandas as pd
import numpy as np
import plotly.express as px
from fastapi.responses import FileResponse, JSONResponse
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Load datasets
taxi_data_path = 'Yellow_Tripdata_2024-10.csv'
zone_data_path = 'transformed_taxi_zone_merged_with_locations.csv'

taxi_data = pd.read_csv(taxi_data_path)
zone_data = pd.read_csv(zone_data_path)

# Create FastAPI app
app = FastAPI()

# Preprocess data
# Merge datasets and extract necessary columns
data = taxi_data.merge(zone_data, left_on='PULocationID', right_on='locationid_x', how='left')
data['pickup_hour'] = pd.to_datetime(data['tpep_pickup_datetime']).dt.hour
data['pickup_day'] = pd.to_datetime(data['tpep_pickup_datetime']).dt.day_name()

def calculate_weekly_demand(day_of_week):
    # Filter data for the selected day
    day_data = data[data['pickup_day'] == day_of_week]

    # Group by zones and calculate metrics
    zone_summary = (
        day_data.groupby('zone')
        .agg({
            'fare_amount': 'mean',
            'trip_distance': 'mean',
            'PULocationID': 'count',
            'zone_latitude': 'mean',
            'zone_longitude': 'mean',
            'pickup_hour': lambda x: x.value_counts().idxmax()  # Most common hour
        })
        .rename(columns={
            'fare_amount': 'avg_earning',
            'trip_distance': 'avg_distance',
            'PULocationID': 'trip_count',
            'pickup_hour': 'peak_hour'
        })
        .reset_index()
    )

    # Sort zones by trip count
    zone_summary = zone_summary.sort_values(by='trip_count', ascending=False)

    # Add Google Maps URL
    zone_summary['maps_url'] = zone_summary.apply(
        lambda row: f"https://www.google.com/maps?q={row['zone_latitude']},{row['zone_longitude']}", axis=1
    )

    return zone_summary[['zone', 'peak_hour', 'avg_earning', 'avg_distance', 'trip_count', 'maps_url']]

def generate_demand_chart(day_of_week, filename='demand_chart.png'):
    day_data = calculate_weekly_demand(day_of_week)
    fig = px.bar(
        day_data,
        x='zone',
        y='trip_count',
        color='avg_earning',
        title=f'Demand on {day_of_week}',
        labels={'trip_count': 'Trip Count', 'avg_earning': 'Average Earning ($)', 'zone': 'Zone'}
    )
    fig.write_image(filename)
    return filename

def train_demand_prediction_model():
    # Prepare data for machine learning
    data['day_of_week'] = pd.to_datetime(data['tpep_pickup_datetime']).dt.dayofweek
    features = data[['pickup_hour', 'day_of_week', 'zone_latitude', 'zone_longitude']]
    target = data['fare_amount']

    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Train a Random Forest Regressor
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

model = train_demand_prediction_model()

@app.get("/weekly-demand")
def get_weekly_demand(day: str):
    """
    Get zones with highest demand for a specific day of the week.
    day: Day of the week (e.g., "Monday", "Tuesday", etc.).
    """
    try:
        if day not in data['pickup_day'].unique():
            return {"status": "error", "message": "Invalid day of the week."}

        demand_data = calculate_weekly_demand(day)
        return {"status": "success", "data": demand_data.to_dict(orient='records')}
    except Exception as e:
        return {"status": "error", "message": str(e)}

@app.get("/weekly-demand/chart")
def get_weekly_demand_chart(day: str):
    """
    Get a chart of weekly demand for a specific day of the week.
    day: Day of the week (e.g., "Monday", "Tuesday", etc.).
    """
    try:
        if day not in data['pickup_day'].unique():
            return {"status": "error", "message": "Invalid day of the week."}

        chart_file = generate_demand_chart(day)
        return FileResponse(chart_file, media_type='image/png', filename='demand_chart.png')
    except Exception as e:
        return {"status": "error", "message": str(e)}

@app.get("/predict")
def predict_demand(pickup_hour: int, day_of_week: int, zone: str):
    """
    Predict demand based on input features.
    pickup_hour: Hour of the day (0-23).
    day_of_week: Day of the week (0=Monday, 6=Sunday).
    zone: Name of the zone.
    """
    try:
        # Get latitude and longitude for the zone
        zone_data = data[data['zone'] == zone].iloc[0]
        latitude = zone_data['zone_latitude']
        longitude = zone_data['zone_longitude']

        # Predict demand
        prediction = model.predict([[pickup_hour, day_of_week, latitude, longitude]])
        return {"status": "success", "predicted_fare": prediction[0]}
    except Exception as e:
        return {"status": "error", "message": str(e)}


C:\Users\jerom\AppData\Local\Temp\ipykernel_21932\1504092631.py:13: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



In [56]:
get_weekly_demand("Sunday")

{'status': 'success',
 'data': [{'zone': 'JFK Airport',
   'peak_hour': 21,
   'avg_earning': 59.07271057565447,
   'avg_distance': 15.539489271578363,
   'trip_count': 28802,
   'maps_url': 'https://www.google.com/maps?q=40.90406018978221,-73.98946405903479'},
  {'zone': 'LaGuardia Airport',
   'peak_hour': 21,
   'avg_earning': 40.8711628875796,
   'avg_distance': 9.389827411771707,
   'trip_count': 16803,
   'maps_url': 'https://www.google.com/maps?q=40.769671120957824,-73.53104754894086'},
  {'zone': 'East Village',
   'peak_hour': 1,
   'avg_earning': 15.267882520978397,
   'avg_distance': 2.564434922335297,
   'trip_count': 16803,
   'maps_url': 'https://www.google.com/maps?q=40.91436875457597,-74.08333419899034'},
  {'zone': 'Upper East Side South',
   'peak_hour': 14,
   'avg_earning': 14.088041857259226,
   'avg_distance': 2.0446738517450966,
   'trip_count': 15959,
   'maps_url': 'https://www.google.com/maps?q=40.82481644952361,-74.00010353638103'},
  {'zone': 'Times Sq/Theat

In [57]:
get_weekly_demand_chart("Sunday")

In [58]:
response = predict_demand(pickup_hour=18, day_of_week=1, zone="JFK Airport")
print(response)



{'status': 'success', 'predicted_fare': 60.05552092001337}


c:\Users\jerom\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names



# Posibles mejoras por realizar. 

1. Integración de Datos en Tiempo Real:

- Incorporar datos de tráfico, clima y eventos locales para ajustar las predicciones y recomendaciones en tiempo real.

2. Predicciones Más Sofisticadas:

- Usar modelos avanzados como XGBoost o redes neuronales para mejorar la exactitud de las predicciones.
- Incorporar variables adicionales como patrones estacionales, datos climáticos, y tendencias económicas.

3. Visualización Interactiva:

- Integrar gráficos y mapas interactivos con herramientas como Plotly, Dash, o Streamlit.

# Ejemplo de implementación de mejora. Modelo 2. 

In [51]:
from fastapi import FastAPI
import pandas as pd
import numpy as np
import plotly.express as px
from fastapi.responses import FileResponse, JSONResponse
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Load datasets
taxi_data_path = 'Yellow_Tripdata_2024-10_reducido.csv'
zone_data_path = 'transformed_taxi_zone_merged_with_locations.csv'

taxi_data = pd.read_csv(taxi_data_path)
zone_data = pd.read_csv(zone_data_path)

# Create FastAPI app
app = FastAPI()

# Preprocess data
data = taxi_data.merge(zone_data, left_on='PULocationID', right_on='locationid_x', how='left')
data['pickup_hour'] = pd.to_datetime(data['tpep_pickup_datetime']).dt.hour
data['pickup_day'] = pd.to_datetime(data['tpep_pickup_datetime']).dt.day_name()
data['day_of_week'] = pd.to_datetime(data['tpep_pickup_datetime']).dt.dayofweek  # 0=Monday, 6=Sunday

# Train XGBoost model
def train_demand_prediction_model():
    # Prepare data for machine learning
    features = data[['pickup_hour', 'day_of_week', 'zone_latitude', 'zone_longitude']]
    target = data['fare_amount']

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Train an XGBoost Regressor
    model = XGBRegressor(n_estimators=200, max_depth=6, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)

    # Print model performance (optional)
    score = model.score(X_test, y_test)
    print(f"Model R^2 Score: {score:.2f}")

    return model

model = train_demand_prediction_model()

@app.get("/predict")
def predict_demand(pickup_hour: int, day_of_week: int, zone: str):
    """
    Predict demand based on input features.
    pickup_hour: Hour of the day (0-23).
    day_of_week: Day of the week (0=Monday, 6=Sunday).
    zone: Name of the zone.
    """
    try:
        # Get latitude and longitude for the zone
        if zone not in data['zone'].unique():
            return {"status": "error", "message": f"Zone '{zone}' not found in data."}

        zone_data = data[data['zone'] == zone].iloc[0]
        latitude = zone_data['zone_latitude']
        longitude = zone_data['zone_longitude']

        # Predict demand
        prediction = model.predict([[pickup_hour, day_of_week, latitude, longitude]])
        return {"status": "success", "predicted_fare": round(prediction[0], 2)}
    except Exception as e:
        return {"status": "error", "message": str(e)}


C:\Users\jerom\AppData\Local\Temp\ipykernel_21932\2270883427.py:13: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



Model R^2 Score: 0.32


In [54]:
response = predict_demand(pickup_hour=18, day_of_week=1, zone="JFK Airport")
print(response)



{'status': 'success', 'predicted_fare': 58.66}


# Cambios Clave
1. Reemplazo del Modelo:

- RandomForestRegressor ha sido reemplazado por XGBRegressor con hiperparámetros iniciales:
    + n_estimators=200: Cantidad de árboles.
    + max_depth=6: Profundidad máxima de los árboles.
    + learning_rate=0.1: Tasa de aprendizaje.
2. Validación del Modelo: Se calcula el coeficiente de determinación 𝑅2 en el conjunto de prueba.
3. Precisión Mejorada: XGBoost generalmente tiene mejor desempeño en conjuntos de datos tabulares y grandes.

# Posibles sobremejoras. 
1. Optimización de Hiperparámetros:
    -   Usa técnicas como búsqueda en cuadrícula (GridSearchCV) o búsqueda aleatoria (RandomizedSearchCV) para encontrar los mejores hiperparámetros.
2. Incorporar Variables Adicionales:
Variables como el clima en tiempo real, eventos locales o estacionalidad podrían mejorar aún más la precisión del modelo.